# Pryngles module: extensions

## External modules

In [1]:
#@external
from pryngles import *

import ctypes
import glob

#Load library
libfile = glob.glob(Misc.get_data('../cpixx*.so'))[0]
cpixx_ext=ctypes.CDLL(libfile)
#@end

## Constants

In [2]:
#@consts:extensions
import ctypes
DOUBLE = ctypes.c_double
PDOUBLE = ctypes.POINTER(DOUBLE)
PPDOUBLE = ctypes.POINTER(PDOUBLE)
PPPDOUBLE = ctypes.POINTER(PPDOUBLE)
#@end:consts

## Fourier coefficients

In [3]:
#@class
class FourierCoefficients(ctypes.Structure):
    _fields_=[
        ("nmat",ctypes.c_int),
        ("nmugs",ctypes.c_int),
        ("nfou",ctypes.c_int),
        ("xmu",PDOUBLE),
        ("rfou",PPPDOUBLE),
        ("rtra",PPPDOUBLE),
    ]
    def __init__(self,nmat,nmugs,nfou,xmu,rfou,rtra):
        self.nmat=nmat
        self.nmugs=nmugs
        self.nfou=nfou
        self.xmu=double1ArrayToPointer(xmu)
        self.rfou=double3ArrayToPointer(rfou)
        self.rtra=double3ArrayToPointer(rtra)
#@end:class

## Specifications cpixx functions

In [11]:
#@standalone:extension

#Sum structure
cpixx_ext.sum_structure.restype = ctypes.c_double
cpixx_ext.sum_structure.argtypes = [ctypes.Structure,
                                    ctypes.c_int,ctypes.c_int,ctypes.c_int]

#Calculate reflection
cpixx_ext.reflection.restype = ctypes.c_int
cpixx_ext.reflection.argtypes = [
    ctypes.Structure,
    ctypes.c_int,
    ctypes.c_int,
    PDOUBLE,PDOUBLE,PDOUBLE,PDOUBLE,PDOUBLE,
    PPDOUBLE
]
#@end:standalone

## Extension utils

In [12]:
#@class
class ExtensionUtil(object):
    """Util routines for extensions.
    """
    def vec2ptr(arr):
        """Converts a 1D numpy to ctypes 1D array. 

        Parameters:
            arr: [ndarray] 1D numpy float64 array

        Return:
            arr_ptr: [ctypes double pointer]
        """
        arr_ptr = arr.ctypes.data_as(PDOUBLE)
        return arr_ptr

    def mat2ptr(arr):
        """ Converts a 2D numpy to ctypes 2D array. 

        Arguments:
            arr: [ndarray] 2D numpy float64 array

        Return:
            arr_ptr: [ctypes double pointer]

        """

        # Init needed data types
        ARR_DIMX = DOUBLE*arr.shape[1]
        ARR_DIMY = PDOUBLE*arr.shape[0]

        # Init pointer
        arr_ptr = ARR_DIMY()

        # Fill the 2D ctypes array with values
        for i, row in enumerate(arr):
            arr_ptr[i] = ARR_DIMX()

            for j, val in enumerate(row):
                arr_ptr[i][j] = val


        return arr_ptr

    def ptr2mat(ptr, n, m):
        """ Converts ctypes 2D array into a 2D numpy array. 

        Arguments:
            arr_ptr: [ctypes double pointer]

        Return:
            arr: [ndarray] 2D numpy float64 array

        """

        arr = np.zeros(shape=(n, m))

        for i in range(n):
            for j in range(m):
                arr[i,j] = ptr[i][j]

        return arr

    def cub2ptr(arr):
        """ Converts a 3D numpy to ctypes 3D array. 

        Arguments:
            arr: [ndarray] 3D numpy float64 array

        Return:
            arr_ptr: [ctypes double pointer]

        """

        # Init needed data types
        ARR_DIMX = DOUBLE*arr.shape[2]
        ARR_DIMY = PDOUBLE*arr.shape[1]
        ARR_DIMZ = PPDOUBLE*arr.shape[0]

        # Init pointer
        arr_ptr = ARR_DIMZ()

        # Fill the 2D ctypes array with values
        for i, row in enumerate(arr):
            arr_ptr[i] = ARR_DIMY()

            for j, col in enumerate(row):
                arr_ptr[i][j] = ARR_DIMX()

                for k, val in enumerate(col):
                    arr_ptr[i][j][k] = val

        return arr_ptr

    def ptr2cub(ptr, n, m, o):
        """ Converts ctypes 3D array into a 3D numpy array. 

        Arguments:
            arr_ptr: [ctypes double pointer]

        Return:
            arr: [ndarray] 3D numpy float64 array

        """

        arr = np.zeros(shape=(n, m, o))

        for i in range(n):
            for j in range(m):
                for k in range(o):
                    arr[i,j,k] = ptr[i][j][k]

        return arr


#@end:class

In [13]:
#@class
class FourierCoefficients(ctypes.Structure):
    """Fourier coefficients ctypes structure
    """
    _fields_=[
        ("nmat",ctypes.c_int),
        ("nmugs",ctypes.c_int),
        ("nfou",ctypes.c_int),
        ("xmu",PDOUBLE),
        ("rfou",PPPDOUBLE),
        ("rtra",PPPDOUBLE),
    ]
    def __init__(self,nmat,nmugs,nfou,xmu,rfou,rtra):
        self.nmat=nmat
        self.nmugs=nmugs
        self.nfou=nfou
        self.xmu=ExtensionUtil.vec2ptr(xmu)
        self.rfou=ExtensionUtil.cub2ptr(rfou)
        self.rtra=ExtensionUtil.cub2ptr(rtra)
#@end:class

In [14]:
#@class
class StokesScatterer(object):
    """Stokes scatterer
    """
    
    def __init__(self,filename):
        self.filename=filename
        self.read_fourier()
        
    def read_fourier(self):
        """
        Read a file containing fourier coefficients produced by PyMieDAP

        Parameters:

           filename: string:

        Returns:

            nmugs: int:
               Number of gaussian integration coefficients.

            nmat: int:
               Number of matrix.

            nfou: int:
               Number of coefficients.

            rfout: array (nmugs*nmat,nmugs,nfou):
               Matrix for the fourier coefficients for reflection.

            rtra: array (nmugs*nmat,nmugs,nfou): 
               Matrix for the fourier coefficients for transmission
        """
        f=open(self.filename)

        #Read header
        nmat=0
        imu=0
        for i,line in enumerate(f):
            if '#' in line:
                continue
            data=line.split()
            if len(data)<3:
                if len(data)==1:
                    if not nmat:
                        nmat=int(data[0])
                    else:
                        nmugs=int(data[0])
                        xmu=np.zeros(nmugs)
                else:
                    xmu[imu]=float(data[0])
                    imu+=1
            else:
                break

        #Get core data
        data=np.loadtxt(self.filename,skiprows=i)
        nfou=int(data[:,0].max())+1

        rfou=np.zeros((nmat*nmugs,nmugs,nfou))
        rtra=np.zeros((nmat*nmugs,nmugs,nfou))

        #Read fourier coefficients
        for row in data:
            m,i,j=int(row[0]),int(row[1])-1,int(row[2])-1
            ibase=i*nmat
            rfou[ibase:ibase+3,j,m]=row[3:3+nmat]
            if len(row[3:])>nmat:
                rtra[ibase:ibase+3,j,m]=row[3+nmat:3+2*nmat]

        verbose(VERB_SIMPLE,f"Checksum '{self.filename}': {rfou.sum()+rtra.sum():.16e}")
        f.close()
        
        self.nmat,self.nmugs,self.nfou=nmat,nmugs,nfou
        self.xmu,self.rfou,self.rtra=xmu,rfou,rtra
        self.F=FourierCoefficients(nmat,nmugs,nfou,xmu,rfou,rtra)
#@end:class

In [15]:
#@test:template
def test_readf(self):
    Verbose.VERBOSITY=VERB_SIMPLE
    
    filename=Misc.get_data("fou_gasplanet.dat")
    S=StokesScatterer(filename)
    Verbose.VERBOSITY=VERB_NONE
    pass

class Test(unittest.TestCase):pass
Test.test_readf=test_readf
unittest.main(argv=['first-arg-is-ignored'],exit=False)
#@end:test

.

  VERB1::read_fourier:: Checksum '/Users/jorgezuluagacallejas/Dropbox/MiInvestigacion/PapersEnProceso/Exorings/pryngles/src/pryngles/data/fou_gasplanet.dat': 5.5117407392827840e+02



----------------------------------------------------------------------
Ran 1 test in 0.035s

OK


In [16]:
#@test:template
def test_fun(self):
    global cpixx_ext
    
    Verbose.VERBOSITY=VERB_SIMPLE
    
    #Test routines
    n=3
    m=3
    p=3
    xmu=np.random.rand(m)
    rfou=np.random.rand(n,m,p)
    rtra=np.random.rand(n,m,p)
    print(rfou)
    print(rfou.sum())
    F=FourierCoefficients(n,m,p,xmu,rfou,rtra)
    suma=cpixx_ext.sum_structure(F,n,m,p)
    print(suma)
    C=ExtensionUtil.ptr2cub(F.rfou,*rfou.shape)
    print(C)
    
    Verbose.VERBOSITY=VERB_NONE
    pass

class Test(unittest.TestCase):pass
Test.test_fun=test_fun
unittest.main(argv=['first-arg-is-ignored'],exit=False)
#@end:test

.

[[[0.15198063 0.95218664 0.78762886]
  [0.20687016 0.7363406  0.10720525]
  [0.77164589 0.58375462 0.42610884]]

 [[0.03853037 0.83688557 0.25565362]
  [0.89395963 0.77567543 0.57889644]
  [0.47047624 0.90793238 0.79352548]]

 [[0.89336131 0.69437689 0.60513714]
  [0.27746254 0.70251864 0.6746486 ]
  [0.58829198 0.6481925  0.98725686]]]
16.34650309696767
16.34650309696767
[[[0.30396126 1.90437327 1.57525772]
  [0.41374033 1.4726812  0.21441049]
  [1.54329179 1.16750923 0.85221769]]

 [[0.07706073 1.67377114 0.51130724]
  [1.78791926 1.55135085 1.15779288]
  [0.94095248 1.81586475 1.58705096]]

 [[1.78672261 1.38875378 1.21027427]
  [0.55492508 1.40503729 1.34929719]
  [1.17658396 1.29638501 1.97451372]]]



----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [27]:
#@method:StokesScatterer
def calculate_stokes(self,phi,beta,theta0,theta,apix,qreflection=1):
    """
    """
    npix=len(phi)
    Sarr=np.zeros((npix,self.F.nmat+1))
    Sarr_ptr=ExtensionUtil.mat2ptr(Sarr)
    cpixx_ext.reflection(self.F,qreflection,npix,
                         ExtensionUtil.vec2ptr(phi),
                         ExtensionUtil.vec2ptr(beta),
                         ExtensionUtil.vec2ptr(theta0),
                         ExtensionUtil.vec2ptr(theta),
                         ExtensionUtil.vec2ptr(apix),
                         Sarr_ptr);
    stokes=ExtensionUtil.ptr2mat(Sarr_ptr,*Sarr.shape)
    return stokes
    
#@end:method

StokesScatterer.calculate_stokes=calculate_stokes

In [31]:
#@test:template
def test_stokes(self):
    global S,phi,beta,theta0,theta,apix
    Verbose.VERBOSITY=VERB_SIMPLE

    #Test values
    npix=1
    phi=np.zeros(npix)
    beta=np.zeros(npix)
    theta0=np.zeros(npix)
    theta=np.zeros(npix)
    apix=np.zeros(npix)

    #######################
    #Planet
    #######################
    #Scatterer
    filename=Misc.get_data("fou_gasplanet.dat")
    S=StokesScatterer(filename)
    
    phi[0]=1.0448451569439827;
    beta[0]=3.069394277348945;
    theta0[0]=0.04990329026929557;
    theta[0]=0.02509670973070432;
    apix[0]=9.432328787795567e-05;
    print("Expected values:\n4.597857424902560283e-07 2.251229972872198058e-07 1.834400800563127439e-09 4.896421313424954569e-01");
    
    #Calculate albedo
    stokes=S.calculate_stokes(phi,beta,theta0,theta,apix)
    print(stokes)
    
    #######################
    #Ring
    #######################
    #Scatterer
    filename=Misc.get_data("fou_ring_0_4_0_8.dat")
    S=StokesScatterer(filename)
    
    #Test values for backscattering
    phi[0]=1.490116119384765625e-08;
    beta[0]=0.000000000000000000e+00;
    theta0[0]=4.999999999999998890e-01;
    theta[0]=5.000000000000000000e-01;
    apix[0]=1.163314390931110409e-04;
    print("Expected:\n6.193646058775441171e-06 -3.046132218287162406e-07 -9.759550180589223642e-15 4.918156751904256135e-02");  

    #Calculate albedo
    stokes=S.calculate_stokes(phi,beta,theta0,theta,apix)
    print(stokes)
    
    #Test values for forwardscattering
    phi[0]=1.601029385538801364e+00;
    beta[0]=1.601029385538801364e+00;
    theta0[0]=1.744974835125044643e-02;
    theta[0]=5.000000000000000000e-01;
    apix[0]=1.163314390931110409e-04;
    print("Expected:\n1.688771016436214060e-07 -1.485273114913191718e-08 2.674513729889046925e-09 8.936444506313186154e-02");  

    #Calculate albedo
    stokes=S.calculate_stokes(phi,beta,theta0,theta,apix,qreflection=0)
    print(stokes)

    Verbose.VERBOSITY=VERB_NONE
    pass

class Test(unittest.TestCase):pass
Test.test_stokes=test_stokes
unittest.main(argv=['first-arg-is-ignored'],exit=False)
#@end:test

  VERB1::read_fourier:: Checksum '/Users/jorgezuluagacallejas/Dropbox/MiInvestigacion/PapersEnProceso/Exorings/pryngles/src/pryngles/data/fou_gasplanet.dat': 5.5117407392827840e+02
Expected values:
4.597857424902560283e-07 2.251229972872198058e-07 1.834400800563127439e-09 4.896421313424954569e-01
[[4.59785742e-07 2.25122997e-07 1.83440080e-09 4.89642131e-01]]


.

  VERB1::read_fourier:: Checksum '/Users/jorgezuluagacallejas/Dropbox/MiInvestigacion/PapersEnProceso/Exorings/pryngles/src/pryngles/data/fou_ring_0_4_0_8.dat': 1.8572277810592641e+03
Expected:
6.193646058775441171e-06 -3.046132218287162406e-07 -9.759550180589223642e-15 4.918156751904256135e-02
[[ 6.19364606e-06 -3.04613222e-07 -9.75955018e-15  4.91815675e-02]]
Expected:
1.688771016436214060e-07 -1.485273114913191718e-08 2.674513729889046925e-09 8.936444506313186154e-02
[[ 1.68877102e-07 -1.48527311e-08  2.67451373e-09  8.93644451e-02]]



----------------------------------------------------------------------
Ran 1 test in 1.027s

OK


In [45]:
#@test:template
def test_stokes_mass(self):
    from time import time
    global S,phi,beta,theta0,theta,apix
    Verbose.VERBOSITY=VERB_SIMPLE

    #Interpolation file
    filename=Misc.get_data("fou_ring_0_4_0_8.dat")
    fileinterp=Misc.get_data("ring-interpolation.mat")
    
    data=np.loadtxt(fileinterp)
    npix=len(data)
    phi=data[:npix,0].copy()
    beta=data[:npix,1].copy()
    theta0=data[:npix,2].copy()
    theta=data[:npix,3].copy()
    apix=data[:npix,7].copy()
    spixx=data[:npix,8:]

    #Scatterer
    S=StokesScatterer(filename)
    
    #Calculate albedo
    st=time()
    stokes=S.calculate_stokes(phi,beta,theta0,theta,apix)
    et=time()
    print("Maximum difference:",abs(spixx-stokes).max())
    print(f"Calculation time (per vector): {(et-st)/npix*1e3} ms")
    
    Verbose.VERBOSITY=VERB_NONE
    pass

class Test(unittest.TestCase):pass
Test.test_stokes_mass=test_stokes_mass
unittest.main(argv=['first-arg-is-ignored'],exit=False)
#@end:test

  VERB1::read_fourier:: Checksum '/Users/jorgezuluagacallejas/Dropbox/MiInvestigacion/PapersEnProceso/Exorings/pryngles/src/pryngles/data/fou_ring_0_4_0_8.dat': 1.8572277810592641e+03


.

Maximum difference: 6.938893903907228e-17
Calculation time (per vector): 0.676600964038403 ms



----------------------------------------------------------------------
Ran 1 test in 1.141s

OK


### The end

In [13]:
#@end:module

## Playing ground

This code is not included in final package files.

In [12]:
f=Foo()